# Quickstart: Pandas API on Spark

This is a short introduction to pandas API on Spark, geared mainly for new users. This notebook shows you some key differences between pandas and pandas API on Spark. You can run this examples by yourself in 'Live Notebook: pandas API on Spark' at [the quickstart page](https://spark.apache.org/docs/latest/api/python/getting_started/index.html).

Customarily, we import pandas API on Spark as follows:

In [3]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

## Object Creation



Creating a pandas-on-Spark Series by passing a list of values, letting pandas API on Spark create a default integer index:

In [4]:
s = ps.Series([1, 3, 5, np.nan, 6, 8])

In [5]:
s

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64

Creating a pandas-on-Spark DataFrame by passing a dict of objects that can be converted to series-like.

In [7]:
psdf = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [8]:
psdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


Creating a pandas DataFrame by passing a numpy array, with a datetime index and labeled columns:

In [9]:
dates = pd.date_range('20130101', periods=6)

In [10]:
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [11]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [12]:
pdf

,A,B,C,D
2013-01-01,-0.389444,-1.746922,0.714097,-0.313742
2013-01-02,-1.140385,1.056478,0.661976,-0.817606
2013-01-03,-0.689304,0.085401,0.707483,0.624892
2013-01-04,-0.328338,0.712479,1.153498,1.012478
2013-01-05,-0.162232,-0.355375,0.203304,0.244954
2013-01-06,1.350330,-0.751248,0.987195,0.201249


Now, this pandas DataFrame can be converted to a pandas-on-Spark DataFrame

In [13]:
psdf = ps.from_pandas(pdf)

In [14]:
type(psdf)

pyspark.pandas.frame.DataFrame

It looks and behaves the same as a pandas DataFrame.

In [15]:
psdf

,A,B,C,D
2013-01-01,-0.389444,-1.746922,0.714097,-0.313742
2013-01-02,-1.140385,1.056478,0.661976,-0.817606
2013-01-03,-0.689304,0.085401,0.707483,0.624892
2013-01-04,-0.328338,0.712479,1.153498,1.012478
2013-01-05,-0.162232,-0.355375,0.203304,0.244954
2013-01-06,1.350330,-0.751248,0.987195,0.201249


Also, it is possible to create a pandas-on-Spark DataFrame from Spark DataFrame easily.

Creating a Spark DataFrame from pandas DataFrame

In [16]:
spark = SparkSession.builder.getOrCreate()

In [17]:
sdf = spark.createDataFrame(pdf)

In [18]:
sdf.show()

+--------------------+--------------------+-------------------+--------------------+
|                   A|                   B|                  C|                   D|
+--------------------+--------------------+-------------------+--------------------+
|-0.38944409782589556| -1.7469220816950644| 0.7140969635229839|-0.31374207001185106|
| -1.1403854250517353|  1.0564780503126945| 0.6619759445665843| -0.8176057910993895|
| -0.6893040260509674| 0.08540087657695608| 0.7074825061244662|  0.6248923719434334|
|-0.32833814862031835|  0.7124794358759834|  1.153497645371343|  1.0124782795790799|
|-0.16223210818657782|-0.35537484585629703|0.20330373680978445| 0.24495414272544072|
|   1.350330463226062| -0.7512481259406727| 0.9871951793235009| 0.20124940425178028|
+--------------------+--------------------+-------------------+--------------------+



Creating pandas-on-Spark DataFrame from Spark DataFrame.

In [19]:
psdf = sdf.to_pandas_on_spark()

In [20]:
psdf

,A,B,C,D
0,-0.389444,-1.746922,0.714097,-0.313742
1,-1.140385,1.056478,0.661976,-0.817606
2,-0.689304,0.085401,0.707483,0.624892
3,-0.328338,0.712479,1.153498,1.012478
4,-0.162232,-0.355375,0.203304,0.244954
5,1.350330,-0.751248,0.987195,0.201249


Having specific [dtypes](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-dtypes) . Types that are common to both Spark and pandas are currently supported.

In [21]:
psdf.dtypes

A    float64
B    float64
C    float64
D    float64
dtype: object

Here is how to show top rows from the frame below.

Note that the data in a Spark dataframe does not preserve the natural order by default. The natural order can be preserved by setting `compute.ordered_head` option but it causes a performance overhead with sorting internally.

In [22]:
psdf.head()

,A,B,C,D
0,-0.389444,-1.746922,0.714097,-0.313742
1,-1.140385,1.056478,0.661976,-0.817606
2,-0.689304,0.085401,0.707483,0.624892
3,-0.328338,0.712479,1.153498,1.012478
4,-0.162232,-0.355375,0.203304,0.244954


Displaying the index, columns, and the underlying numpy data.

In [23]:
psdf.index

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [24]:
psdf.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [25]:
psdf.to_numpy()

array([[-0.3894441 , -1.74692208,  0.71409696, -0.31374207],
       [-1.14038543,  1.05647805,  0.66197594, -0.81760579],
       [-0.68930403,  0.08540088,  0.70748251,  0.62489237],
       [-0.32833815,  0.71247944,  1.15349765,  1.01247828],
       [-0.16223211, -0.35537485,  0.20330374,  0.24495414],
       [ 1.35033046, -0.75124813,  0.98719518,  0.2012494 ]])

Showing a quick statistic summary of your data

In [26]:
psdf.describe()

,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,-0.226562,-0.166531,0.737925,0.158704
std,0.845792,1.020497,0.324937,0.652977
min,-1.140385,-1.746922,0.203304,-0.817606
25%,-0.689304,-0.751248,0.661976,-0.313742
50%,-0.389444,-0.355375,0.707483,0.201249
75%,-0.162232,0.712479,0.987195,0.624892
max,1.350330,1.056478,1.153498,1.012478


Transposing your data

In [27]:
psdf.T

,0,1,2,3,4,5
A,-0.389444,-1.140385,-0.689304,-0.328338,-0.162232,1.350330
B,-1.746922,1.056478,0.085401,0.712479,-0.355375,-0.751248
C,0.714097,0.661976,0.707483,1.153498,0.203304,0.987195
D,-0.313742,-0.817606,0.624892,1.012478,0.244954,0.201249


Sorting by its index

In [28]:
psdf.sort_index(ascending=False)

,A,B,C,D
5,1.350330,-0.751248,0.987195,0.201249
4,-0.162232,-0.355375,0.203304,0.244954
3,-0.328338,0.712479,1.153498,1.012478
2,-0.689304,0.085401,0.707483,0.624892
1,-1.140385,1.056478,0.661976,-0.817606
0,-0.389444,-1.746922,0.714097,-0.313742


Sorting by value

In [29]:
psdf.sort_values(by='B')

,A,B,C,D
0,-0.389444,-1.746922,0.714097,-0.313742
5,1.350330,-0.751248,0.987195,0.201249
4,-0.162232,-0.355375,0.203304,0.244954
2,-0.689304,0.085401,0.707483,0.624892
3,-0.328338,0.712479,1.153498,1.012478
1,-1.140385,1.056478,0.661976,-0.817606


## Missing Data
Pandas API on Spark primarily uses the value `np.nan` to represent missing data. It is by default not included in computations. 


In [30]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])

In [31]:
pdf1.loc[dates[0]:dates[1], 'E'] = 1

In [32]:
psdf1 = ps.from_pandas(pdf1)

In [33]:
psdf1

,A,B,C,D,E
2013-01-01,-0.389444,-1.746922,0.714097,-0.313742,1.0
2013-01-02,-1.140385,1.056478,0.661976,-0.817606,1.0
2013-01-03,-0.689304,0.085401,0.707483,0.624892,NaN
2013-01-04,-0.328338,0.712479,1.153498,1.012478,NaN


To drop any rows that have missing data.

In [34]:
psdf1.dropna(how='any')

,A,B,C,D,E
2013-01-01,-0.389444,-1.746922,0.714097,-0.313742,1.0
2013-01-02,-1.140385,1.056478,0.661976,-0.817606,1.0


Filling missing data.

In [35]:
psdf1.fillna(value=5)

,A,B,C,D,E
2013-01-01,-0.389444,-1.746922,0.714097,-0.313742,1.0
2013-01-02,-1.140385,1.056478,0.661976,-0.817606,1.0
2013-01-03,-0.689304,0.085401,0.707483,0.624892,5.0
2013-01-04,-0.328338,0.712479,1.153498,1.012478,5.0


## Operations

### Stats
Performing a descriptive statistic:

In [36]:
psdf.mean()

A   -0.226562
B   -0.166531
C    0.737925
D    0.158704
dtype: float64

### Spark Configurations

Various configurations in PySpark could be applied internally in pandas API on Spark.
For example, you can enable Arrow optimization to hugely speed up internal pandas conversion. See also <a href="https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html">PySpark Usage Guide for Pandas with Apache Arrow</a> in PySpark documentation.

In [37]:
prev = spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")  # Keep its default value.
ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [39]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
%timeit ps.range(300000).to_pandas()

Py4JError: An error occurred while calling o1285.arrowPySparkSelfDestructEnabled. Trace:
py4j.Py4JException: Method arrowPySparkSelfDestructEnabled([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Unknown Source)



In [40]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", False)
%timeit ps.range(300000).to_pandas()

778 ms ± 31.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
ps.reset_option("compute.default_index_type")
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", prev)  # Set its default value back.

## Grouping
By “group by” we are referring to a process involving one or more of the following steps:

- Splitting the data into groups based on some criteria
- Applying a function to each group independently
- Combining the results into a data structure

In [42]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})

In [43]:
psdf

,A,B,C,D
0,foo,one,-0.196284,0.072564
1,bar,one,-0.906931,-0.404891
2,foo,two,-1.029691,-0.178795
3,bar,three,-0.273997,-0.136076
4,foo,two,1.461260,-0.172588
5,bar,two,-1.880659,1.824222
6,foo,one,0.916974,-0.804587
7,foo,three,-0.162186,2.662183


Grouping and then applying the [sum()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.groupby.GroupBy.sum.html) function to the resulting groups.

In [44]:
psdf.groupby('A').sum()

,C,D
A,,
bar,-3.061586,1.283255
foo,0.990074,1.578776


Grouping by multiple columns forms a hierarchical index, and again we can apply the sum function.

In [45]:
psdf.groupby(['A', 'B']).sum()

C         D
A   B                        
foo one    0.720690 -0.732024
    two    0.431569 -0.351383
bar three -0.273997 -0.136076
foo three -0.162186  2.662183
bar two   -1.880659  1.824222
    one   -0.906931 -0.404891

## Plotting

In [46]:
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))

In [47]:
psser = ps.Series(pser)

In [48]:
psser = psser.cummax()

In [56]:
psser.plot()

On a DataFrame, the [plot()](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.plot.html) method is a convenience to plot all of the columns with labels:

In [51]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])

In [52]:
psdf = ps.from_pandas(pdf)

In [53]:
psdf = psdf.cummax()

In [54]:
psdf.plot()

For more details, [Plotting](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/frame.html#plotting) documentation.

## Getting data in/out

### CSV

CSV is straightforward and easy to use. See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_csv.html) to write a CSV file and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_csv.html) to read a CSV file.

In [57]:
psdf.to_csv('foo.csv')
ps.read_csv('foo.csv').head(10)

Py4JJavaError: An error occurred while calling o2212.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at 

### Parquet

Parquet is an efficient and compact file format to read and write faster. See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_paruqet.html) to write a Parquet file and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_parquet.html) to read a Parquet file.

In [58]:
psdf.to_parquet('bar.parquet')
ps.read_parquet('bar.parquet').head(10)

Py4JJavaError: An error occurred while calling o2220.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at 

### Spark IO

In addition, pandas API on Spark fully supports Spark's various datasources such as ORC and an external datasource.  See [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.to_orc.html) to write it to the specified datasource and [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.read_orc.html) to read it from the datasource.

In [59]:
psdf.to_spark_io('zoo.orc', format="orc")
ps.read_spark_io('zoo.orc', format="orc").head(10)

Py4JJavaError: An error occurred while calling o2228.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at 

See the [Input/Output](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/io.html) documentation for more details.